# Making sense of the world through vision

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
import numpy as np
import sys
import time
import os
import IPython
#from IPython.display import Image 
from array import array
#from PIL import Image as pilImage
from skimage import io
from io import BytesIO
from mpl_toolkits.axes_grid1 import AxesGrid
from IPython.display import display
from IPython.display import Image as IPythonImage

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, ImageFileCreateBatch
from msrest.authentication import ApiKeyCredentials

import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person

In [ ]:
subscription_key = "<INSERT KEY>"
endpoint = "https://westeurope.api.cognitive.microsoft.com/" 

# 




# Computer vision

In [ ]:
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

## Describe what is on an image

In [ ]:
image_url = "images/amsterdam-streets-2.jpg"

In [ ]:
display(IPythonImage(filename=image_url))

In [ ]:
with open(os.path.join(image_url), "rb") as image_stream:
    description_results = computervision_client.describe_image_in_stream(image_stream)

    for description in description_results.captions:
        print(description.text, "| Confidence: ","%.2f" % description.confidence)

## Detect what is on the image

In [ ]:
# Detect objects in the Images
with open(os.path.join(image_url), "rb") as image_stream:
    detect_objects_results_remote = computervision_client.detect_objects_in_stream(image_stream)

    im = plt.imread(image_url)

    # Create figure and axes
    fig = plt.figure(figsize = (im.shape[1]/80, im.shape[0]/80))
    ax = plt.axes((0,0,1,1))

    # Display the image
    ax.imshow(im,origin='upper')

    # Overlay the information
    for object in detect_objects_results_remote.objects:
        color = (np.random.rand(),np.random.rand(),np.random.rand())
        rect = patches.Rectangle((object.rectangle.x, object.rectangle.y), 
                                 object.rectangle.w, object.rectangle.h, 
                                 linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)

        ax.text(
            (1/im.shape[1]*object.rectangle.x), 1-(1/im.shape[0]*object.rectangle.y), object.object_property,
            horizontalalignment='left',
            verticalalignment='bottom',
            fontsize=16,
            color='w',
            backgroundcolor=color,
            transform=ax.transAxes
        )
    ax.axis('off')
    plt.show()

In [ ]:
bank_note_img = "bank-notes/Test/2Thousandnote/3.jpg"

with open(os.path.join(bank_note_img), "rb") as image_stream:
    
    # detect objects in the image
    detect_objects_results_remote = computervision_client.detect_objects_in_stream(image_stream)

    im = plt.imread(bank_note_img)

    # Create figure and axes
    fig = plt.figure(figsize = (im.shape[1]/100, im.shape[0]/100))
    ax = plt.axes((0,0,1,1))

    # Display the image
    ax.imshow(im,origin='upper')

    # Overlay the information
    for object in detect_objects_results_remote.objects:
        color = (np.random.rand(),np.random.rand(),np.random.rand())
        rect = patches.Rectangle((object.rectangle.x, object.rectangle.y), 
                                 object.rectangle.w, object.rectangle.h, 
                                 linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)

        ax.text(
            (1/im.shape[1]*object.rectangle.x), 1-(1/im.shape[0]*object.rectangle.y), object.object_property,
            horizontalalignment='left',
            verticalalignment='bottom',
            fontsize=16,
            color='w',
            backgroundcolor=color,
            transform=ax.transAxes
        )
    ax.axis('off')
    plt.show()

# 




# Detect your own objects




## Train our own model using Azure Custom Vision

In [ ]:
# Training credentials
training_key = "<INSERT KEY>"
cv_endpoint = "https://westeurope.api.cognitive.microsoft.com"

# Prediction credentials
prediction_key = "<INSERT KEY>"
prediction_resource_id = "/subscriptions/431dbae5-40ca-438a-8daf-77d7d5580b41/resourceGroups/CustomVision_Demo-rg/providers/Microsoft.CognitiveServices/accounts/CustomVisionDemo-Prediction"

# Location for the training images
training_images = "bank-notes/train"

In [ ]:
# Connect to the training endpoint
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint=cv_endpoint,credentials=credentials)

In [ ]:
for domain in trainer.get_domains():
    print(domain.id, "\t", domain.name) 

In [ ]:
# Create a new project using the standard domain
project = trainer.create_project("Bank Notes - 2", domain_id="ee85a74c-405e-4adc-bb47-ffa8ca0c9f31")

In [ ]:
image_list = []
directories = os.listdir(training_images)

for tagName in directories:
    tag = trainer.create_tag(project.id, tagName)
    images = os.listdir(os.path.join(training_images,tagName))
    for img in images:
        with open(os.path.join(training_images,tagName,img), "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=img, contents=image_contents.read(), tag_ids=[tag.id]))  
            
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]
batchedImages = chunks(image_list, 64)

for batchOfImages in batchedImages:
    upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=batchOfImages))
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
    else:
        print("Batch uploaded successfully")
print("Done uploading")

In [ ]:
# Show the dataset
path = r"bank-notes/train"
random_filenames = []
for tag in os.listdir(path):
    random_filenames.append(path+"/"+tag+"/"+random.choice([
        x for x in os.listdir(os.path.join(path,tag))
        if os.path.isfile(os.path.join(path,tag, x))
    ]))

grid = AxesGrid(plt.figure(1, (20,20)), 111, nrows_ncols=(2, 4), axes_pad=0, label_mode="1")

i = 0
for img_name in random_filenames[0:10]:
    im = plt.imread(img_name)
    grid[i].imshow(im,aspect='auto', extent=(0,1,0,0.8), alpha=1, origin='upper', zorder=-1)
    i = i+1

In [ ]:
# Train the model
print ("Start Training...")
iteration = trainer.train_project(project.id)

In [ ]:
# Check for completion
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 30 seconds...")
    time.sleep(30)
print("Training complete")

In [ ]:
# Publish the model
publish_iteration_name = "version1"

print("Publishing the model")
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Publishing complete")

In [ ]:
# Now there is a trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(cv_endpoint, prediction_credentials)

# Pick random images from the test set
path = r"bank-notes/Test"
random_filenames = []
for tag in os.listdir(path):
    random_filenames.append(path+"/"+tag+"/"+random.choice([
        x for x in os.listdir(os.path.join(path,tag))
        if os.path.isfile(os.path.join(path,tag, x))
    ]))
grid = AxesGrid(plt.figure(1, (20,20)), 111, nrows_ncols=(4, 2), axes_pad=0, label_mode="1")

# Send the images one by one to the model and show the result
i = 0
for img_name in random_filenames[0:8]:
        
    with open(os.path.join (img_name), "rb") as image_contents:
        results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())
        
        im = plt.imread(img_name)
        grid[i].imshow(im,aspect='auto', extent=(0,1,0,0.5), alpha=1, origin='upper', zorder=-1), 
            
        grid[i].text(0.05,0.05,
                     results.predictions[0].tag_name +  ": {0:.2f}%".format(results.predictions[0].probability * 100),
                     fontsize=30,
                     backgroundcolor='w')
    
    i = i+1


# 





# Who is on the image?

In [ ]:
image = "images/prime-minister.jpg"
pil_img = IPythonImage(image)
display(pil_img)

In [ ]:
with open(os.path.join(image), "rb") as image_stream:
    detect_faces = computervision_client.analyze_image_in_stream(image_stream, ["faces"])

In [ ]:
im = plt.imread(image)

# Create figure and axes
fig = plt.figure(figsize = (im.shape[1]/100, im.shape[0]/100))
ax = plt.axes((0,0,1,1))

# Display the image
ax.imshow(im,origin='upper')

# Overlay the information
for face in detect_faces.faces:
    color = (np.random.rand(),np.random.rand(),np.random.rand())
    rect = patches.Rectangle((face.face_rectangle.left, face.face_rectangle.top), 
                             face.face_rectangle.width, face.face_rectangle.height, 
                             linewidth=6, edgecolor=color, facecolor='none')
    ax.add_patch(rect)

    ax.text(
        (1/im.shape[1]*face.face_rectangle.left), 1-(1/im.shape[0]*face.face_rectangle.top), 
        "{} of age {}".format(face.gender,face.age),
        horizontalalignment='left',
        verticalalignment='bottom',
        fontsize=44,
        color='w',
        backgroundcolor=color,
        transform=ax.transAxes
    )
ax.axis('off')
plt.show()

In [ ]:
with open(os.path.join(image), "rb") as image_stream:
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream("celebrities",image_stream)

In [ ]:
im = plt.imread(image)

# Create figure and axes
fig = plt.figure(figsize = (im.shape[1]/100, im.shape[0]/100))
ax = plt.axes((0,0,1,1))

# Display the image
ax.imshow(im,origin='upper')

# Overlay the information
for face in detect_domain_results_celebs_remote.result["celebrities"]:
    color = (np.random.rand(),np.random.rand(),np.random.rand())
    rect = patches.Rectangle((face['faceRectangle']['left'], face['faceRectangle']['top']), 
                             face['faceRectangle']['width'], face['faceRectangle']['height'], linewidth=6, edgecolor=color, facecolor='none')
    ax.add_patch(rect)
   
    ax.text(
        (1/im.shape[1]*face['faceRectangle']['left']), 1-(1/im.shape[0]*face['faceRectangle']['top']), 
        "{}".format(face['name']),
        horizontalalignment='left',
        verticalalignment='bottom',
        fontsize=44,
        color='w',
        backgroundcolor=color,
        transform=ax.transAxes
    )
ax.axis('off')
plt.show()

# 





# Get more insights on Faces

[Face API](https://docs.microsoft.com/azure/cognitive-services/face/)

In [ ]:
face_api_key = "<INSERT KEY>"
face_api_endpoint = "https://azconf-face.cognitiveservices.azure.com/"

In [ ]:
# Create an authenticated FaceClient.
face_client = FaceClient(face_api_endpoint, CognitiveServicesCredentials(face_api_key))

In [ ]:
mf_image = "face-photos/modern_family.jpg"
display(IPythonImage(filename=mf_image))

In [ ]:
with open(os.path.join(mf_image), "rb") as image_stream:
    
    # Detect faces in images
    detected_faces = face_client.face.detect_with_stream(image_stream, return_face_attributes=[
                    'age',  # Could have been the string 'age'
                    'gender',
                    'smile',
                    'facialHair',
                    'glasses',
                    'emotion',
                    'hair',
                    'makeup',
                    'accessories'
                ])
    
    # Display the result
    pil_img = Image.open(mf_image)
    for face in detected_faces: 
        img2 = pil_img.crop((face.face_rectangle.left, face.face_rectangle.top, face.face_rectangle.left+face.face_rectangle.width, face.face_rectangle.top+face.face_rectangle.height))
        display(img2)
        print (f'Face id: {face.face_id}')
        print (f'Gender: {face.face_attributes.gender}')
        print (f'smile: {face.face_attributes.smile}')
        print (f'age: {face.face_attributes.age}')
        print (f'facial_hair moustache: {face.face_attributes.facial_hair.moustache}')
        print (f'facial_hair beard: {face.face_attributes.facial_hair.beard}')
        print (f'facial_hair sideburns: {face.face_attributes.facial_hair.sideburns}')
        print (f'glasses: {face.face_attributes.glasses}')
        print (f'eye_makeup: {face.face_attributes.makeup.eye_makeup}')
        print (f'lip_makeup: {face.face_attributes.makeup.lip_makeup}')
        print (f'emotion: {face.face_attributes.emotion}')
        print(" ==")

    print()

# Save this ID for use in Find Similar
first_image_face_ID = detected_faces[0].face_id

## Train the face API to recognize people

In [ ]:
# Show the dataset
path = r"face-photos/train"
random_filenames = []
for train_img in os.listdir(path):
    random_filenames.append(os.path.join(path, train_img))

grid = AxesGrid(plt.figure(1, (20,20)), 111, nrows_ncols=(1, 5), axes_pad=0, label_mode="1")

i = 0
for img_name in random_filenames[0:10]:
    im = plt.imread(img_name)
    grid[i].imshow(im,aspect='auto', extent=(0,0.8,0,1), alpha=1, origin='upper', zorder=-1)
    i = i+1

In [ ]:
PERSON_GROUP_ID = "modern-family"
face_client.person_group.delete(person_group_id=PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

In [ ]:
path = r"face-photos/train"

for person in os.listdir(path):
    name = person.partition(".")[0]
    print("Adding:"+name)
    w = open(os.path.join(path,person), 'r+b')

    # Create a person
    person = face_client.person_group_person.create(PERSON_GROUP_ID, name)

    # Add a face to the person
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, person.person_id, w)

In [ ]:
# Train the person group
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        face_client.person_group.delete(person_group_id=PERSON_GROUP_ID)
        sys.exit('Training the person group has failed.')
    time.sleep(5)

## Identify people in the image

In [ ]:
with open(os.path.join(mf_image), "rb") as image_stream:
    # Detect faces
    face_ids = []
    # We use detection model 3 to get better performance.
    faces = face_client.face.detect_with_stream(image_stream, detection_model='detection_03')
    for face in faces:
        face_ids.append(face.face_id)
        print(f'found face: {face.face_id}')

In [ ]:
# Identify faces
results = face_client.face.identify(face_ids, PERSON_GROUP_ID)

identified_persons = {}

for person in results:
    for candidate in person.candidates:
        identified_person = face_client.person_group_person.get(PERSON_GROUP_ID,candidate.person_id)
        print("Found: "+identified_person.name)
        identified_persons[person.face_id] = identified_person

In [ ]:
# Show the result

im = plt.imread(mf_image)

# Create figure and axes
fig = plt.figure(figsize = (im.shape[1]/70, im.shape[0]/70))
ax = plt.axes((0,0,1,1))

# Display the image
ax.imshow(im,origin='upper')

# Overlay the information
for face in faces:
    color = (np.random.rand(),np.random.rand(),np.random.rand())
    rect = patches.Rectangle((face.face_rectangle.left, face.face_rectangle.top), 
                             face.face_rectangle.width, face.face_rectangle.height, 
                             linewidth=3, edgecolor=color, facecolor='none')
    ax.add_patch(rect)

    ax.text(
        (1/im.shape[1]*face.face_rectangle.left), 1-(1/im.shape[0]*face.face_rectangle.top), 
        "{}".format(identified_persons[face.face_id].name),
        horizontalalignment='left', verticalalignment='bottom', fontsize=16, color='w', backgroundcolor=color, transform=ax.transAxes
    )
ax.axis('off')
plt.show()

# 




## Read text in images

In [ ]:
read_image_url = "images/satya-acc.jpeg"

In [ ]:
display(IPythonImage(filename=read_image_url))

In [ ]:
print("===== Start =====")

# Call API with URL and raw response (allows you to get the operation location)
with open(os.path.join(read_image_url), "rb") as image_stream:
    read_response = computervision_client.read_in_stream(image_stream,  raw=True)

    read_operation_location = read_response.headers["Operation-Location"]
    # Grab the ID from the URL
    operation_id = read_operation_location.split("/")[-1]

    # Call the "GET" API and wait for it to retrieve the results 
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)
        
print("===== Done =====")

In [ ]:
im = plt.imread(read_image_url)

# Create figure and axes
fig = plt.figure(figsize = (im.shape[1]/100, im.shape[0]/100))
ax = plt.axes((0,0,1,1))

# Display the image
ax.imshow(im,origin='upper')

for text_result in read_result.analyze_result.read_results:
    for line in text_result.lines:
        color = (np.random.rand(),np.random.rand(),np.random.rand())
        rect = patches.Rectangle((line.bounding_box[0], line.bounding_box[1]), 
                             line.bounding_box[2]-line.bounding_box[0], line.bounding_box[7]-line.bounding_box[1], linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        print(line.text)
    
ax.axis('off')
plt.show()

In [ ]:
handwriting_image_url = "images/handwriting.jpg"

In [ ]:
display(IPythonImage(filename=handwriting_image_url))

In [ ]:
print("===== Start =====")
# Call API with URL and raw response (allows you to get the operation location)
with open(os.path.join(handwriting_image_url), "rb") as image_stream:
    read_response = computervision_client.read_in_stream(image_stream,  raw=True)

read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

print("===== Done =====")

In [ ]:
im = plt.imread(handwriting_image_url)

# Create figure and axes
fig = plt.figure(figsize = (im.shape[1]/100, im.shape[0]/100))
ax = plt.axes((0,0,1,1))

# Display the image
ax.imshow(im,origin='upper')

for text_result in read_result.analyze_result.read_results:
    for line in text_result.lines:
        color = (np.random.rand(),np.random.rand(),np.random.rand())
        rect = patches.Rectangle((line.bounding_box[0], line.bounding_box[1]), 
                             line.bounding_box[2]-line.bounding_box[0], line.bounding_box[5]-line.bounding_box[1], 
                             linewidth=6, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        print(line.text)
    
ax.axis('off')
plt.show()

# Recap



## Computer Vision
- Describe images
- Detect objects
- Detect faces
- Detrect celebrities 

## Custom Vision
- Trained our own model to classify images

## Face API
- Describe faces in images
- Trained our own person group
- Identified persons in an image

